In [140]:
import json
import tqdm
from flair.data import Sentence

In [10]:
def read_data(input_file_path):
    data = []

    with open(input_file_path) as f:
        for line in f:
            data.append(json.loads(line))

    return data

In [11]:
def write_data(output_file_path):
    # Open a new JSON file for writing
    with open(output_file_path, "w") as output_file:
        for data_line in data:
            output_file.write(json.dumps(data_line) + "\n")

    print(f"Data with 'Test' property added has been saved to {output_file_path}")

In [387]:
data = read_data("./squad_data_validation_pos_ner.json")

In [456]:
def find_start_position(text, pos_context, answer, context):
    sentence = Sentence(text)
    tokens = []
    spaces_no = context[:answer].count(" ")

    for word in sentence:
        tokens.append(word.text)

    start_position = 0
    token_number = 0
    match = False
    no_match = -1
    for start_position in range(len(pos_context) - len(tokens)):
        k_words_context = [list(line.values())[0][0] for line in 
                      pos_context[start_position:start_position + len(tokens)]]
        next_word = [list(line.values())[0][0] for line in 
                      pos_context[start_position + len(tokens):start_position + len(tokens) + 1]]
        if k_words_context == tokens and (token_number + spaces_no == answer or token_number + spaces_no + 1 == answer):
            match = True
            break
        
        if (token_number + spaces_no == answer):
            no_match = start_position
            break
        elif next_word == ' ' and token_number + spaces_no + 1 == answer:
            no_match = start_position
            break
                
        token_number += len(k_words_context[0])
    if match:
        return start_position
    return no_match

In [459]:
special_characters = ["$", ",", "''", "-LRB-", "-RRB-", ".", ":", "``"]


def get_proc_pos_between_item(pos_context, context, answers):
    positions = {}
    copy_answer = []
    i = 0
    for answer in answers["answer_start"]:
        answer_position = find_start_position(answers["text"][i], pos_context, answer, context)
        if answer_position != -1:
            positions[answer_position] = answers["answer_start"][i]
            copy_answer.append(answers["answer_start"][i])
        i += 1
    pos = ""
    offset = 0
    offsets = {}
    for index in range(len(pos_context)):
        pos_text = pos_context[index][str(index)]

        if pos_text[1] in special_characters:
            pos_text[1] = "sym"
        if index in positions.keys():
            offsets[positions[index]] = offset
        pos += f"{pos_text[0]} [{pos_text[1].lower()}] "
        offset += len(f" [{pos_text[1]}] ")

    answer_starts = []

    for index in range(len(copy_answer)):
        answer = copy_answer[index]
        spaces_no = context[:answer].count(" ") + 1        
        answer += offsets[answer] - spaces_no
        if pos[answer : answer + 1] == " ":
            answer += 1
        answer_starts.append(answer)

    return pos, answer_starts


In [460]:
example_id = 65

pos, answer_starts = get_proc_pos_between_item(data[example_id]['POS_context'], data[example_id]['context'],
                          data[example_id]['answers'])
pos

"The [dt] Broncos [nnps] took [vbd] an [dt] early [jj] lead [nn] in [in] Super [nnp] Bowl [nnp] 50 [cd] and [cc] never [rb] trailed [vbd] . [sym] Newton [nnp] was [vbd] limited [vbn] by [in] Denver [nnp] 's [pos] defense [nn] , [sym] which [wdt] sacked [vbd] him [prp] seven [cd] times [nns] and [cc] forced [vbd] him [prp] into [in] three [cd] turnovers [nns] , [sym] including [vbg] a [dt] fumble [nn] which [wdt] they [prp] recovered [vbd] for [in] a [dt] touchdown [nn] . [sym] Denver [nnp] linebacker [nn] Von [nnp] Miller [nnp] was [vbd] named [vbn] Super [nnp] Bowl [nnp] MVP [nnp] , [sym] recording [vbg] five [cd] solo [jj] tackles [nns] , [sym] 2 [cd] ½ [sym] sacks [nns] , [sym] and [cc] two [cd] forced [vbn] fumbles [nns] . [sym] "

In [476]:
for item in tqdm.tqdm(data):
    context = item['context']
    question = item['question']
    item['context'], item['answers']['answer_start'] = get_proc_pos_between_item(item['POS_context'], context, item['answers'])

    item.pop('POS_context', None)
    item.pop('POS_question', None)
    item.pop('NER_question', None)
    item.pop('NER_context', None)

100%|██████████| 10570/10570 [00:09<00:00, 1150.60it/s]


In [477]:
write_data("./squad_data_validation_pos_between.json")

Data with 'Test' property added has been saved to ./squad_data_validation_pos_between.json


In [478]:
proc_data = read_data('./squad_data_validation_pos_between.json')

In [479]:
proc_data

[{'id': '56be4db0acb8001400a502ec',
  'title': 'Super_Bowl_50',
  'context': 'Super [nnp] Bowl [nnp] 50 [cd] was [vbd] an [dt] American [jj] football [nn] game [nn] to [to] determine [vb] the [dt] champion [nn] of [in] the [dt] National [nnp] Football [nnp] League [nnp] ( [hyph] NFL [nnp] ) [sym] for [in] the [dt] 2015 [cd] season [nn] . [sym] The [dt] American [nnp] Football [nnp] Conference [nnp] ( [sym] AFC [nnp] ) [sym] champion [nn] Denver [nnp] Broncos [nnps] defeated [vbd] the [dt] National [nnp] Football [nnp] Conference [nnp] ( [sym] NFC [nnp] ) [sym] champion [nn] Carolina [nnp] Panthers [nnps] 24 [cd] – [sym] 10 [cd] to [to] earn [vb] their [prp$] third [jj] Super [nnp] Bowl [nnp] title [nn] . [sym] The [dt] game [nn] was [vbd] played [vbn] on [in] February [nnp] 7 [cd] , [sym] 2016 [cd] , [sym] at [in] Levi [nnp] \'s [pos] Stadium [nnp] in [in] the [dt] San [nnp] Francisco [nnp] Bay [nnp] Area [nnp] at [in] Santa [nnp] Clara [nnp] , [sym] California [nnp] . [sym] As [in] th